#**US Passengers Data Pre-processing**

In [ ]:
#import drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#set directory
import os
os.chdir('/content/drive/MyDrive/thesis-folder')

In [ ]:
#import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy as sc
import datetime
import calendar

In [ ]:
!pip install geopy 
!pip install Nominatim

  Created wheel for Nominatim: filename=nominatim-0.1-cp37-none-any.whl size=2365 sha256=e088131ddeb2ab4a48590c27f45029eb618573ddca3ca81b907c5b8b2b9d5ba2
  Stored in directory: /root/.cache/pip/wheels/d5/66/ed/e7476981dc30210b6b5ce7c25b054e8db35d44fdd2198003d4
Successfully built Nominatim


#Functions

In [ ]:
#retrieve address, latitude and longitude by giving both city and country names
def return_loc_by_city_country(city_name, country_name):
  from  geopy.geocoders import Nominatim
  geolocator = Nominatim()
  city =city_name
  country = country_name
  loc = geolocator.geocode(city+','+ country)
  #print(loc.address)
  #print("latitude is :" ,loc.latitude,"\nlongtitude is:" ,loc.longitude)
  return loc.address, loc.latitude, loc.longitude


#retrieve address, latitude and longitude by giving only the city name
def return_loc_by_city(city_name): 
  from geopy.geocoders import Nominatim
  address=city_name
  geolocator = Nominatim(user_agent="Your_Name")
  location = geolocator.geocode(address)
  #print(location.address)
  #print((location.latitude, location.longitude)) 
  return location.address, location.latitude, location.longitude

#pre-process inconsistent data format in city and country
def pre_process_airport_code(bts_code,backfilled_code):

  city = []
  country = []

  bts_code['city_airport'] = bts_code.city_airport.str.split(':', expand=True)[0]

  for index, row in bts_code.iterrows():
    city_country = row.city_airport.split(',')
    city.append(city_country[0])
    if len(row.city_airport.split(',')) <= 2 :
      country.append(city_country[1])

    else:
      address = return_loc_by_city(city_country[0])[0]
      country.append(address.split(',')[-1])
      #print(row.apt_code, city_country[0], address.split(',')[-1])

  bts_code['city'] = city
  bts_code['country'] = country

  '''if flag == 'us_airports':
    bts_code.rename(columns={'apt_code':'usg_apt', 'city':'usg_city', 'country':'usg_country'}, inplace=True)
    backfilled_code.rename(columns={'apt_code':'usg_apt', 'city':'usg_city', 'country':'usg_country'}, inplace=True)
  
  else:
    bts_code.rename(columns={'apt_code':'fg_apt', 'city':'fg_city', 'country':'fg_country'}, inplace=True)
    backfilled_code.rename(columns={'apt_code':'fg_apt', 'city':'fg_city', 'country':'fg_country'}, inplace=True)'''

  bts_code.drop(columns='city_airport', inplace=True)

  bts_code = pd.concat([bts_code,backfilled_code], axis=0, ignore_index=True)

  return bts_code

#Read Files


In [ ]:
#International US passengers data
us_df = pd.read_csv('dataset/US air passengers 1990-2020.csv')
us_df

,data_dte,Year,Month,usg_apt_id,usg_apt,usg_wac,fg_apt_id,fg_apt,fg_wac,airlineid,carrier,carriergroup,type,Scheduled,Charter,Total
0,06/01/2007,2007,6,13204,MCO,33,16085,YHZ,951,20364,C6,0,Passengers,0,683,683
1,12/01/2005,2005,12,11433,DTW,43,10411,AUA,277,20344,RD,1,Passengers,0,1010,1010
2,04/01/2003,2003,4,13487,MSP,63,16304,ZIH,148,20204,MG,1,Passengers,0,508,508
3,12/01/2005,2005,12,12016,GUM,5,11138,CRK,766,20312,TZ,1,Passengers,0,76,76
4,03/01/2007,2007,3,14843,SJU,3,15084,SXM,259,20421,SLQ,1,Passengers,0,35,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682775,10/01/2015,2015,10,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,130217,0,130217
682776,09/01/2014,2014,9,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,132052,0,132052
682777,10/01/2014,2014,10,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,132822,0,132822
682778,08/01/2014,2014,8,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,134263,0,134263


In [ ]:
#file containing airport code and their corresponding country details
transport_codes_df = pd.read_excel('dataset/bureau-of-transportation-stats-data.xlsx')
transport_codes_df

,apt_code,city_airport
0,01A,"Afognak Lake, AK: Afognak Lake Airport"
1,03A,"Granite Mountain, AK: Bear Creek Mining Strip"
2,04A,"Lik, AK: Lik Mining Camp"
3,05A,"Little Squaw, AK: Little Squaw Airport"
4,06A,"Kizhuyak, AK: Kizhuyak Bay"
...,...,...
6504,ZXY,"Delta, CO: Blake Field"
6505,ZXZ,"Waterville, WA: Waterville Airport"
6506,ZYL,"Sylhet, Bangladesh: Savannakhet"
6507,ZZU,"Mzuzu, Malawi: Mzuzu Airport"


In [ ]:
#codes that are left without country details
backfilled_codes_df = pd.read_excel('dataset/backfilled-missing-codes.xlsx')
backfilled_codes_df

,apt_code,city,country
0,NQZ,Akmola,Kazakhstan
1,BS1,Bristol,United Kingdom
2,BS3,Bristol,United Kingdom
3,BS2,Bristol,United Kingdom
4,BYJ,Beja,Portugal
5,GKE,Geilenkirchen,Germany
6,ZMH,British Columbia,Canada
7,OLL,Oyo,Congo
8,3TX,Decatur,USA


In [ ]:
#file with latitude and longitude details for countries
fg_lat_long_df1 = pd.read_csv('dataset/foreign_lat_long_mapping.csv')
fg_lat_long_df1

,foreign_city,foreign_country,fg_lat,fg_long
0,acapulco,mexico,16.868049,-99.894018
1,calgary,canada,51.046095,-114.065465
2,cancun,mexico,21.161785,-86.851047
3,edmonton,canada,53.535411,-113.507996
4,guatemala city,guatemala,14.622233,-90.518519
...,...,...,...,...
1217,heraklion,greece,35.340013,25.134348
1218,taranto,italy,40.548197,17.081155
1219,cranfield,united kingdom,52.069021,-0.607932
1220,san jose,philippines,121.880200,12.084861


In [ ]:
#file with latitude and longitude for countries
fg_lat_long_df2 = pd.read_csv('dataset/backfilled_fg_lat_long.csv')
fg_lat_long_df2

,foreign_city,foreign_country,fg_lat,fg_long
0,achutupo,panama,9.1983,-77.9873
1,mamai,papua new guinea,25.1858,86.6750
2,tonga tapu,tonga,-21.1466,-175.2515
3,isle baltra,ecuador,-0.4597,-90.2714
4,kobe honshu island,japan,34.6901,135.1956
5,lhok sukon,indonesia,5.3553,95.2418
6,nambaiyufa,papua new guinea,-6.2500,145.2330
7,big bay marina,canada,50.3833,-125.1330
8,naberevnye chelny,russia,55.7306,52.3893
9,ganges harbor,canada,48.8556,-123.4965


#Data Pre-processing

**Computed columns**

1. us_city
2. foreign_city
3. foreign_country
4. us_latitude
5. us_longitude
6. foreign_lat
7. foreign_long
8. month


In [ ]:
#split data into city and country
airport_code_df = pre_process_airport_code(transport_codes_df, backfilled_codes_df)
airport_code_df

,apt_code,city,country
0,01A,Afognak Lake,AK
1,03A,Granite Mountain,AK
2,04A,Lik,AK
3,05A,Little Squaw,AK
4,06A,Kizhuyak,AK
...,...,...,...
6513,BYJ,Beja,Portugal
6514,GKE,Geilenkirchen,Germany
6515,ZMH,British Columbia,Canada
6516,OLL,Oyo,Congo


In [ ]:
#merge pax data with US airport code data
#merged_us_df = us_df.merge(us_code_df, on=['usg_apt'], how='left')
merged_us_df = us_df.merge(airport_code_df, left_on=['usg_apt'], right_on=['apt_code'], how='left')
merged_us_df.drop(columns='apt_code', inplace=True)
merged_us_df

,data_dte,Year,Month,usg_apt_id,usg_apt,usg_wac,fg_apt_id,fg_apt,fg_wac,airlineid,carrier,carriergroup,type,Scheduled,Charter,Total,city,country
0,06/01/2007,2007,6,13204,MCO,33,16085,YHZ,951,20364,C6,0,Passengers,0,683,683,Orlando,FL
1,12/01/2005,2005,12,11433,DTW,43,10411,AUA,277,20344,RD,1,Passengers,0,1010,1010,Detroit,MI
2,04/01/2003,2003,4,13487,MSP,63,16304,ZIH,148,20204,MG,1,Passengers,0,508,508,Minneapolis,MN
3,12/01/2005,2005,12,12016,GUM,5,11138,CRK,766,20312,TZ,1,Passengers,0,76,76,Guam,TT
4,03/01/2007,2007,3,14843,SJU,3,15084,SXM,259,20421,SLQ,1,Passengers,0,35,35,San Juan,PR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682775,10/01/2015,2015,10,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,130217,0,130217,New York,NY
682776,09/01/2014,2014,9,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,132052,0,132052,New York,NY
682777,10/01/2014,2014,10,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,132822,0,132822,New York,NY
682778,08/01/2014,2014,8,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,134263,0,134263,New York,NY


In [ ]:
#merge pax data with foreign airport code data
us_pax_df = merged_us_df.merge(airport_code_df, left_on=['fg_apt'], right_on=['apt_code'], how='left')
#us_pax_df['foreign_country'] = merged_us_df.merge(airport_code_df, left_on=['fg_apt'], right_on=['apt_code'], how='left')['country']
us_pax_df.rename(columns={'city_x':'us_city', 'city_y':'foreign_city', 'country_y':'foreign_country'}, inplace=True)
us_pax_df.drop(columns=['country_x','apt_code'], inplace=True)
us_pax_df

,data_dte,Year,Month,usg_apt_id,usg_apt,usg_wac,fg_apt_id,fg_apt,fg_wac,airlineid,carrier,carriergroup,type,Scheduled,Charter,Total,us_city,foreign_city,foreign_country
0,06/01/2007,2007,6,13204,MCO,33,16085,YHZ,951,20364,C6,0,Passengers,0,683,683,Orlando,Halifax,Canada
1,12/01/2005,2005,12,11433,DTW,43,10411,AUA,277,20344,RD,1,Passengers,0,1010,1010,Detroit,Aruba,Aruba
2,04/01/2003,2003,4,13487,MSP,63,16304,ZIH,148,20204,MG,1,Passengers,0,508,508,Minneapolis,Ixtapa/Zihuatanejo,Mexico
3,12/01/2005,2005,12,12016,GUM,5,11138,CRK,766,20312,TZ,1,Passengers,0,76,76,Guam,Luzon Island,Philippines
4,03/01/2007,2007,3,14843,SJU,3,15084,SXM,259,20421,SLQ,1,Passengers,0,35,35,San Juan,Philipsburg,Sint Maarten
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682775,10/01/2015,2015,10,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,130217,0,130217,New York,London,United Kingdom
682776,09/01/2014,2014,9,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,132052,0,132052,New York,London,United Kingdom
682777,10/01/2014,2014,10,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,132822,0,132822,New York,London,United Kingdom
682778,08/01/2014,2014,8,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,134263,0,134263,New York,London,United Kingdom


In [ ]:
#add computed column-month
us_pax_df['month'] = pd.to_datetime(us_pax_df['Month'], format='%m').dt.month_name()
us_pax_df

,data_dte,Year,Month,usg_apt_id,usg_apt,usg_wac,fg_apt_id,fg_apt,fg_wac,airlineid,carrier,carriergroup,type,Scheduled,Charter,Total,us_city,foreign_city,foreign_country,month
0,06/01/2007,2007,6,13204,MCO,33,16085,YHZ,951,20364,C6,0,Passengers,0,683,683,Orlando,Halifax,Canada,June
1,12/01/2005,2005,12,11433,DTW,43,10411,AUA,277,20344,RD,1,Passengers,0,1010,1010,Detroit,Aruba,Aruba,December
2,04/01/2003,2003,4,13487,MSP,63,16304,ZIH,148,20204,MG,1,Passengers,0,508,508,Minneapolis,Ixtapa/Zihuatanejo,Mexico,April
3,12/01/2005,2005,12,12016,GUM,5,11138,CRK,766,20312,TZ,1,Passengers,0,76,76,Guam,Luzon Island,Philippines,December
4,03/01/2007,2007,3,14843,SJU,3,15084,SXM,259,20421,SLQ,1,Passengers,0,35,35,San Juan,Philipsburg,Sint Maarten,March
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682775,10/01/2015,2015,10,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,130217,0,130217,New York,London,United Kingdom,October
682776,09/01/2014,2014,9,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,132052,0,132052,New York,London,United Kingdom,September
682777,10/01/2014,2014,10,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,132822,0,132822,New York,London,United Kingdom,October
682778,08/01/2014,2014,8,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,134263,0,134263,New York,London,United Kingdom,August


In [ ]:
us_pax_df['us_city'] = us_pax_df['us_city'].str.lower()
us_pax_df['foreign_city'] = us_pax_df['foreign_city'].str.lower()
us_pax_df['foreign_country'] = us_pax_df['foreign_country'].str.lower()

In [ ]:
#checking whether there are any missing city and country values
us_pax_df.isna().sum()

data_dte              0
Year                  0
Month                 0
usg_apt_id            0
usg_apt               0
usg_wac               0
fg_apt_id             0
fg_apt                0
fg_wac                0
airlineid             0
carrier            2797
carriergroup          0
type                  0
Scheduled             0
Charter               0
Total                 0
us_city             411
foreign_city          0
foreign_country       0
month                 0
dtype: int64

In [ ]:
#print airport codes with missing city values
print(len(us_pax_df[us_pax_df.us_city.isna()].usg_apt.unique()))
us_unique_list = []
us_unique_list = list(us_pax_df[us_pax_df.us_city.isna()].usg_apt.unique())
print(us_unique_list)

1
['ZZZ']


In [ ]:
#records with missing values
zzz_df = us_pax_df[us_pax_df['usg_apt'] == 'ZZZ']
zzz_df

,data_dte,Year,Month,usg_apt_id,usg_apt,usg_wac,fg_apt_id,fg_apt,fg_wac,airlineid,carrier,carriergroup,type,Scheduled,Charter,Total,us_city,foreign_city,foreign_country,month
295,05/01/2005,2005,5,99999,ZZZ,1,13162,MAJ,844,20344,RD,1,Passengers,0,35,35,NaN,majuro,marshall islands,May
412,05/01/2008,2008,5,99999,ZZZ,1,10790,BUD,437,20344,RD,1,Passengers,0,127,127,NaN,budapest,hungary,May
707,09/01/2009,2009,9,99999,ZZZ,1,16217,YUL,941,21080,04Q,1,Passengers,0,4,4,NaN,montreal,canada,September
1934,07/01/2008,2008,7,99999,ZZZ,1,14681,SAP,131,20344,RD,1,Passengers,0,140,140,NaN,san pedro sula,honduras,July
1955,03/01/2006,2006,3,99999,ZZZ,1,13604,NAP,450,20344,RD,1,Passengers,0,88,88,NaN,naples,italy,March
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142026,08/01/2014,2014,8,99999,ZZZ,1,16123,YMP,906,20272,KAH,1,Passengers,30,0,30,NaN,port mcneill,canada,August
142218,07/01/2014,2014,7,99999,ZZZ,1,16076,YHH,906,20272,KAH,1,Passengers,31,0,31,NaN,campbell river,canada,July
145634,08/01/2014,2014,8,99999,ZZZ,1,16601,BC3,906,20272,KAH,1,Passengers,44,0,44,NaN,bliss landing,canada,August
148570,07/01/2014,2014,7,99999,ZZZ,1,16310,ZNA,906,20272,KAH,1,Passengers,53,58,111,NaN,nanaimo,canada,July


Dropping ZZZ airport code rows because it belongs to Aarhus sea airport, located in Denmark.It might be updated recently by IATA.

In [ ]:
#drop ZZZ airport code from the master data
us_pax_df = us_pax_df[us_pax_df['usg_apt'] != 'ZZZ']
us_pax_df.shape

(682369, 20)

##Cleaning the data

In [ ]:
#drop unwanted columns
#columns dropped - usg_apt_id, usg_wac, fg_wac, airlineid, carrier, carriergroup, type, usg_apt, fg_apt
us_pax_df.drop(columns=['usg_apt_id','usg_wac','fg_apt_id','fg_wac','airlineid','carrier','carriergroup','type','usg_apt','fg_apt'], inplace=True)

#rename columns
us_pax_df.rename(columns={'data_dte':'date', 'Month':'month_num', 'Year':'year', 'Scheduled':'scheduled','Charter':'charter','Total':'total'}, inplace=True)

#convert to lower case
us_pax_df['us_city'] = us_pax_df['us_city'].str.lower()
us_pax_df['foreign_city'] = us_pax_df['foreign_city'].str.lower()
us_pax_df['foreign_country'] = us_pax_df['foreign_country'].str.lower()
us_pax_df['month'] = us_pax_df['month'].str.lower()

#convert date to datetime datatype
#us_pax_df['date'] = pd.to_datetime(us_pax_df['date'],format='%m/%d/%Y').dt.strftime('%d/%m/%Y')
us_pax_df['date'] = pd.to_datetime(us_pax_df['date'])

#order the dataframe columns
us_pax_df = us_pax_df[['us_city','foreign_city','foreign_country','month','month_num','year','scheduled','charter','total']]

#remove spaces
us_pax_df.foreign_country = us_pax_df.foreign_country.apply(lambda x: x.lstrip())
us_pax_df.foreign_country = us_pax_df.foreign_country.apply(lambda x: x.split('(')[0])

#removing city aliases by removing the content after "/"
us_pax_df['us_city'] = us_pax_df['us_city'].str.split('/',expand=True)[0]
us_pax_df['foreign_city'] = us_pax_df['foreign_city'].str.split('/', expand=True)[0]

us_pax_df

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__

,us_city,foreign_city,foreign_country,month,month_num,year,scheduled,charter,total
0,orlando,halifax,canada,june,6,2007,0,683,683
1,detroit,aruba,aruba,december,12,2005,0,1010,1010
2,minneapolis,ixtapa,mexico,april,4,2003,0,508,508
3,guam,luzon island,philippines,december,12,2005,0,76,76
4,san juan,philipsburg,sint maarten,march,3,2007,0,35,35
...,...,...,...,...,...,...,...,...,...
682775,new york,london,united kingdom,october,10,2015,130217,0,130217
682776,new york,london,united kingdom,september,9,2014,132052,0,132052
682777,new york,london,united kingdom,october,10,2014,132822,0,132822
682778,new york,london,united kingdom,august,8,2014,134263,0,134263


##Retrieving latitude and longitude using Geopy and Nominatim

In [ ]:
'''
#Mapping latitude and longitude of 1200 unique foreign cities
i = 0
problematic_city_loc = []
unique_city_df['fg_lat'] = ''
unique_city_df['fg_long'] = ''

for index, row in unique_city_df.iterrows():
    try:
      row['fg_lat'] = return_loc_by_city_country(row['foreign_city'], row['foreign_country'])[-2]
      row['fg_long'] = return_loc_by_city_country(row['foreign_city'], row['foreign_country'])[-1]
      i+=1
      print('Retrieving location for {} th record'.format(i))
    
    
    except AttributeError:
        problematic_city_loc.append(row['foreign_city'])
        

unique_city_df.to_csv('foreign_lat_long_mapping.csv', index=False)
'''

"\n#Mapping latitude and longitude of 1200 unique foreign cities\ni = 0\nproblematic_city_loc = []\nunique_city_df['fg_lat'] = ''\nunique_city_df['fg_long'] = ''\n\nfor index, row in unique_city_df.iterrows():\n    try:\n      row['fg_lat'] = return_loc_by_city_country(row['foreign_city'], row['foreign_country'])[-1]\n      row['fg_long'] = return_loc_by_city_country(row['foreign_city'], row['foreign_country'])[-2]\n      i+=1\n      print('Retrieving location for {} th record'.format(i))\n    \n    \n    except AttributeError:\n        problematic_city_loc.append(row['foreign_city'])\n        \n\nunique_city_df.to_csv('foreign_lat_long_mapping.csv', index=False)\n"

In [ ]:
'''
us_cities_list = us_df.us_city.unique()
us_city = []
us_lat = []
us_long = []
problematic_city = []
i = 1

try:
  for city in us_cities_list:
    us_city.append(city)
    us_lat.append(return_loc_by_city_country(city, 'united states')[1])
    us_long.append(return_loc_by_city_country(city, 'united states')[2])
    print('Retrieving {}th record'.format(i))
    i += 1

except AttributeError:
  problematic_city.append(city)

city_dict = {'us_city': us_city, 'us_lat': us_lat, 'us_long': us_long}
us_lat_long_df = pd.DataFrame(data=city_dict)

us_lat_long_df.to_csv('dataset/us_lat_long.csv', index=False)
'''

"\nus_cities_list = us_df.us_city.unique()\nus_city = []\nus_lat = []\nus_long = []\nproblematic_city = []\ni = 1\n\ntry:\n  for city in us_cities_list:\n    us_city.append(city)\n    us_lat.append(return_loc_by_city_country(city, 'united states')[1])\n    us_long.append(return_loc_by_city_country(city, 'united states')[2])\n    print('Retrieving {}th record'.format(i))\n    i += 1\n\nexcept AttributeError:\n  problematic_city.append(city)\n\ncity_dict = {'us_city': us_city, 'us_lat': us_lat, 'us_long': us_long}\nus_lat_long_df = pd.DataFrame(data=city_dict)\n\nus_lat_long_df.to_csv('dataset/us_lat_long.csv', index=False)\n"

In [ ]:
#combine foreign cities' latitude and longitude into single dataframe from two files
fg_lat_long_df = pd.concat([fg_lat_long_df1,fg_lat_long_df2], axis=0, ignore_index=True)
fg_lat_long_df.rename(columns={'fg_lat':'foreign_lat', 'fg_long':'foreign_long'}, inplace=True)
fg_lat_long_df

,foreign_city,foreign_country,foreign_lat,foreign_long
0,acapulco,mexico,16.868049,-99.894018
1,calgary,canada,51.046095,-114.065465
2,cancun,mexico,21.161785,-86.851047
3,edmonton,canada,53.535411,-113.507996
4,guatemala city,guatemala,14.622233,-90.518519
...,...,...,...,...
1238,tauta,papua new guinea,-5.850000,145.917000
1239,andenes,norway,69.316100,16.120200
1240,princess lousia inlet,canada,50.181200,-123.813400
1241,andeness,norway,69.316100,16.120200


In [ ]:
#join foreign cities' latitude and longitude details with the master dataframe
us_df = pd.merge(us_pax_df, fg_lat_long_df, how='left', on=['foreign_city','foreign_country'])
us_df.dropna(axis=0, inplace=True)
us_df.reset_index(inplace=True, drop=True)
us_df

,us_city,foreign_city,foreign_country,month,month_num,year,scheduled,charter,total,foreign_lat,foreign_long
0,orlando,halifax,canada,june,6,2007,0,683,683,44.648618,-63.585949
1,detroit,aruba,aruba,december,12,2005,0,1010,1010,12.490300,-69.960984
2,minneapolis,ixtapa,mexico,april,4,2003,0,508,508,16.807623,-92.898650
3,guam,luzon island,philippines,december,12,2005,0,76,76,12.750349,122.731210
4,san juan,philipsburg,sint maarten,march,3,2007,0,35,35,18.025071,-63.048307
...,...,...,...,...,...,...,...,...,...,...,...
682364,new york,london,united kingdom,october,10,2015,130217,0,130217,51.507322,-0.127647
682365,new york,london,united kingdom,september,9,2014,132052,0,132052,51.507322,-0.127647
682366,new york,london,united kingdom,october,10,2014,132822,0,132822,51.507322,-0.127647
682367,new york,london,united kingdom,august,8,2014,134263,0,134263,51.507322,-0.127647


In [ ]:
#read US latitude and longitude from file
us_cities_df = pd.read_csv('dataset/us_lat_long.csv')
us_cities_df

,us_city,us_lat,us_long
0,orlando,28.542111,-81.379030
1,detroit,42.331551,-83.046640
2,minneapolis,44.977300,-93.265469
3,guam,13.450126,144.757551
4,san juan,18.465299,-66.116666
...,...,...,...
662,limestone,31.526185,-96.580617
663,kapolei,21.334008,-158.077849
664,chatham,39.676163,-89.704544
665,king salmon,58.755161,-156.519247


In [ ]:
#combine US cities' latitude and longitude with master dataframe
us_df = pd.merge(us_df, us_cities_df, how='left', on=['us_city'])
us_df

,us_city,foreign_city,foreign_country,month,month_num,year,scheduled,charter,total,foreign_lat,foreign_long,us_lat,us_long
0,orlando,halifax,canada,june,6,2007,0,683,683,44.648618,-63.585949,28.542111,-81.379030
1,detroit,aruba,aruba,december,12,2005,0,1010,1010,12.490300,-69.960984,42.331551,-83.046640
2,minneapolis,ixtapa,mexico,april,4,2003,0,508,508,16.807623,-92.898650,44.977300,-93.265469
3,guam,luzon island,philippines,december,12,2005,0,76,76,12.750349,122.731210,13.450126,144.757551
4,san juan,philipsburg,sint maarten,march,3,2007,0,35,35,18.025071,-63.048307,18.465299,-66.116666
...,...,...,...,...,...,...,...,...,...,...,...,...,...
682364,new york,london,united kingdom,october,10,2015,130217,0,130217,51.507322,-0.127647,40.712728,-74.006015
682365,new york,london,united kingdom,september,9,2014,132052,0,132052,51.507322,-0.127647,40.712728,-74.006015
682366,new york,london,united kingdom,october,10,2014,132822,0,132822,51.507322,-0.127647,40.712728,-74.006015
682367,new york,london,united kingdom,august,8,2014,134263,0,134263,51.507322,-0.127647,40.712728,-74.006015


In [ ]:
#cleaning df2 and replacing the correct city name for faulty city names
us_df[us_df.foreign_city == 'isle baltra'].foreign_city = 'isla baltra'
us_df[us_df.foreign_city == 'kobe honshu island'].foreign_city = 'kobe'
us_df[us_df.foreign_city == 'lhok sukon'].foreign_city = 'lhok seudu'
us_df[us_df.foreign_city == 'naberevnye chelny'].foreign_city = 'naberezhnye chelny'
us_df[us_df.foreign_city == 'sungei tekai'].foreign_city = 'sungai tekali'
us_df[us_df.foreign_city == 'andeness'].foreign_city = 'andenes'

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
#group the dataset by month and year
agg_dict = {'month_num':'first', 'scheduled':'sum', 'charter':'sum','total':'sum',  'us_lat':'first', 'us_long':'first','foreign_lat':'first', 'foreign_long':'first'}
#us_grouped_df = us_df.groupby(['year','month','us_city','foreign_city','foreign_country'], as_index=False).agg(agg_dict)
us_grouped_df = us_df.groupby(['year','month','us_city','foreign_city','foreign_country'], as_index=False).agg(agg_dict)
us_grouped_df = us_grouped_df[['us_city', 'foreign_city','foreign_country','year','month','month_num','us_lat','us_long', 'foreign_lat', 'foreign_long', 'scheduled', 'charter', 'total']]
#us_pax_grouped_df.reset_index(inplace=True)
us_grouped_df

,us_city,foreign_city,foreign_country,year,month,month_num,us_lat,us_long,foreign_lat,foreign_long,scheduled,charter,total
0,albany,montreal,canada,1990,april,4,42.651167,-73.754968,45.497216,-73.610364,0,183,183
1,allentown,freeport,the bahamas,1990,april,4,40.602206,-75.471279,26.535681,-78.695362,0,30,30
2,anchorage,amsterdam,netherlands,1990,april,4,61.216313,-149.894852,52.372760,4.893604,10007,0,10007
3,anchorage,brussels,belgium,1990,april,4,61.216313,-149.894852,50.843735,4.359779,3254,0,3254
4,anchorage,copenhagen,denmark,1990,april,4,61.216313,-149.894852,55.686724,12.570072,5739,0,5739
...,...,...,...,...,...,...,...,...,...,...,...,...,...
436235,washington,sofia,bulgaria,2020,may,5,38.894992,-77.036558,42.697863,23.322179,0,609,609
436236,washington,toronto,canada,2020,may,5,38.894992,-77.036558,43.653482,-79.383935,235,0,235
436237,west palm beach,saguenay,canada,2020,may,5,26.715364,-80.053294,48.405959,-71.069183,0,1,1
436238,westhampton,london,united kingdom,2020,may,5,40.830312,-72.662574,51.507322,-0.127647,0,10,10


Drop 2020's records

In [ ]:
#check shape of the dataframe before dropping
us_grouped_df.shape

(436240, 13)

In [ ]:
#drop 2020 records as it tends to show variation in patterns due to pandemic
us_grouped_df= us_grouped_df[us_grouped_df['year'] != 2020]
us_grouped_df

,us_city,foreign_city,foreign_country,year,month,month_num,us_lat,us_long,foreign_lat,foreign_long,scheduled,charter,total
0,albany,montreal,canada,1990,april,4,42.651167,-73.754968,45.497216,-73.610364,0,183,183
1,allentown,freeport,the bahamas,1990,april,4,40.602206,-75.471279,26.535681,-78.695362,0,30,30
2,anchorage,amsterdam,netherlands,1990,april,4,61.216313,-149.894852,52.372760,4.893604,10007,0,10007
3,anchorage,brussels,belgium,1990,april,4,61.216313,-149.894852,50.843735,4.359779,3254,0,3254
4,anchorage,copenhagen,denmark,1990,april,4,61.216313,-149.894852,55.686724,12.570072,5739,0,5739
...,...,...,...,...,...,...,...,...,...,...,...,...,...
430002,white plains,naples,italy,2019,september,9,41.033986,-73.762910,40.835934,14.248783,0,14,14
430003,white plains,paris,france,2019,september,9,41.033986,-73.762910,48.856697,2.351462,0,2,2
430004,white plains,sonderborg,denmark,2019,september,9,41.033986,-73.762910,54.908919,9.789800,0,1,1
430005,white plains,toronto,canada,2019,september,9,41.033986,-73.762910,43.653482,-79.383935,0,19,19


In [ ]:
#verify whether the dataframe has 2020's records after dropping
us_grouped_df[us_grouped_df['year'] == 2020]

,us_city,foreign_city,foreign_country,year,month,month_num,us_lat,us_long,foreign_lat,foreign_long,scheduled,charter,total


In [ ]:
#there are 58629 records for the years 2017,2018,2019
us_grouped_df[us_grouped_df['year'].isin([2017,2018,2019])]

,us_city,foreign_city,foreign_country,year,month,month_num,us_lat,us_long,foreign_lat,foreign_long,scheduled,charter,total
371131,abilene,calgary,canada,2017,april,4,32.446450,-99.747591,-114.065465,51.046095,62,0,62
371132,abilene,monterrey,mexico,2017,april,4,32.446450,-99.747591,-100.315258,25.680202,59,0,59
371133,abilene,queretaro,mexico,2017,april,4,32.446450,-99.747591,-99.847560,20.854258,61,0,61
371134,albany,toronto,canada,2017,april,4,42.651167,-73.754968,-79.383935,43.653482,0,3,3
371135,albuquerque,guatemala city,guatemala,2017,april,4,35.084103,-106.650985,-90.518519,14.622233,0,503,503
...,...,...,...,...,...,...,...,...,...,...,...,...,...
430002,white plains,naples,italy,2019,september,9,41.033986,-73.762910,14.248783,40.835934,0,14,14
430003,white plains,paris,france,2019,september,9,41.033986,-73.762910,2.351462,48.856697,0,2,2
430004,white plains,sonderborg,denmark,2019,september,9,41.033986,-73.762910,9.789800,54.908919,0,1,1
430005,white plains,toronto,canada,2019,september,9,41.033986,-73.762910,-79.383935,43.653482,0,19,19


In [ ]:
#check missing values
us_grouped_df.isna().sum()

us_city            0
foreign_city       0
foreign_country    0
year               0
month              0
month_num          0
us_lat             0
us_long            0
foreign_lat        0
foreign_long       0
scheduled          0
charter            0
total              0
dtype: int64

In [ ]:
#save the resultant data
us_grouped_df.to_csv('intermittent-result/International US/US_pax_processed.csv', index=False)

#Generate Season-wise data

In [ ]:
#generate a new dataframe for summer season pax data
us_summer_df = us_grouped_df[us_grouped_df.month.isin(['june', 'july', 'august'])]
us_summer_df

,us_city,foreign_city,foreign_country,year,month,month_num,us_lat,us_long,foreign_lat,foreign_long,scheduled,charter,total
790,aguadilla,port of spain,trinidad and tobago,1990,august,8,18.449655,-67.118476,-61.518017,10.657268,0,467,467
791,albany,montreal,canada,1990,august,8,42.651167,-73.754968,-73.610364,45.497216,45,0,45
792,albany,toronto,canada,1990,august,8,42.651167,-73.754968,-79.383935,43.653482,91,0,91
793,anchorage,amsterdam,netherlands,1990,august,8,61.216313,-149.894852,4.893604,52.372760,15672,0,15672
794,anchorage,brussels,belgium,1990,august,8,61.216313,-149.894852,4.359779,50.843735,4120,0,4120
...,...,...,...,...,...,...,...,...,...,...,...,...,...
421955,white plains,pristina,kosovo,2019,june,6,41.033986,-73.762910,21.164085,42.663877,0,7,7
421956,white plains,rome,italy,2019,june,6,41.033986,-73.762910,12.482932,41.893320,0,2,2
421957,white plains,shannon,ireland,2019,june,6,41.033986,-73.762910,-8.878837,52.710454,0,4,4
421958,white plains,toronto,canada,2019,june,6,41.033986,-73.762910,-79.383935,43.653482,0,16,16


In [ ]:
#check whether data is present only for summer months
us_summer_df.month.unique()

array(['august', 'july', 'june'], dtype=object)

In [ ]:
#generate a new dataframe for autumn season pax data
us_autumn_df = us_grouped_df[us_grouped_df.month.isin(['september', 'october', 'november'])]
us_autumn_df

,us_city,foreign_city,foreign_country,year,month,month_num,us_lat,us_long,foreign_lat,foreign_long,scheduled,charter,total
7412,aguadilla,providenciales,turks and caicos islands,1990,november,11,18.449655,-67.118476,-72.300446,21.802061,0,50,50
7413,aguadilla,st. lucia,saint lucia,1990,november,11,18.449655,-67.118476,-60.975036,13.825049,0,123,123
7414,albany,cancun,mexico,1990,november,11,42.651167,-73.754968,-86.851047,21.161785,0,330,330
7415,allentown,freeport,the bahamas,1990,november,11,40.602206,-75.471279,-78.695362,26.535681,0,138,138
7416,anchorage,amsterdam,netherlands,1990,november,11,61.216313,-149.894852,4.893604,52.372760,5265,0,5265
...,...,...,...,...,...,...,...,...,...,...,...,...,...
430002,white plains,naples,italy,2019,september,9,41.033986,-73.762910,14.248783,40.835934,0,14,14
430003,white plains,paris,france,2019,september,9,41.033986,-73.762910,2.351462,48.856697,0,2,2
430004,white plains,sonderborg,denmark,2019,september,9,41.033986,-73.762910,9.789800,54.908919,0,1,1
430005,white plains,toronto,canada,2019,september,9,41.033986,-73.762910,-79.383935,43.653482,0,19,19


In [ ]:
#check whether data is present only for autumn months
us_autumn_df.month.unique()

array(['november', 'october', 'september'], dtype=object)

In [ ]:
#generate a new dataframe for winter season pax data
us_winter_df = us_grouped_df[us_grouped_df.month.isin(['december','january','february'])]
us_winter_df

,us_city,foreign_city,foreign_country,year,month,month_num,us_lat,us_long,foreign_lat,foreign_long,scheduled,charter,total
1636,albany,montreal,canada,1990,december,12,42.651167,-73.754968,-73.610364,45.497216,108,0,108
1637,anchorage,amsterdam,netherlands,1990,december,12,61.216313,-149.894852,4.893604,52.372760,6028,0,6028
1638,anchorage,copenhagen,denmark,1990,december,12,61.216313,-149.894852,12.570072,55.686724,4559,0,4559
1639,anchorage,dusseldorf,germany,1990,december,12,61.216313,-149.894852,6.776314,51.225402,1736,0,1736
1640,anchorage,london,united kingdom,1990,december,12,61.216313,-149.894852,-0.127647,51.507322,16032,0,16032
...,...,...,...,...,...,...,...,...,...,...,...,...,...
418382,white plains,toronto,canada,2019,january,1,41.033986,-73.762910,-79.383935,43.653482,0,4,4
418383,wilmington,goose bay,canada,2019,january,1,39.745947,-75.546589,-60.416667,53.333333,0,2,2
418384,wilmington,guatemala city,guatemala,2019,january,1,39.745947,-75.546589,-90.518519,14.622233,0,2,2
418385,wilmington,providenciales,turks and caicos islands,2019,january,1,39.745947,-75.546589,-72.300446,21.802061,0,4,4


In [ ]:
#check whether data is present only for winter months
us_winter_df.month.unique()

array(['december', 'february', 'january'], dtype=object)

In [ ]:
#generate a new dataframe for spring season pax data
us_spring_df = us_grouped_df[us_grouped_df.month.isin(['march', 'april', 'may'])]
us_spring_df

,us_city,foreign_city,foreign_country,year,month,month_num,us_lat,us_long,foreign_lat,foreign_long,scheduled,charter,total
0,albany,montreal,canada,1990,april,4,42.651167,-73.754968,-73.610364,45.497216,0,183,183
1,allentown,freeport,the bahamas,1990,april,4,40.602206,-75.471279,-78.695362,26.535681,0,30,30
2,anchorage,amsterdam,netherlands,1990,april,4,61.216313,-149.894852,4.893604,52.372760,10007,0,10007
3,anchorage,brussels,belgium,1990,april,4,61.216313,-149.894852,4.359779,50.843735,3254,0,3254
4,anchorage,copenhagen,denmark,1990,april,4,61.216313,-149.894852,12.570072,55.686724,5739,0,5739
...,...,...,...,...,...,...,...,...,...,...,...,...,...
425343,white plains,st. petersburg,russia,2019,may,5,41.033986,-73.762910,30.316229,59.938732,0,2,2
425344,white plains,toronto,canada,2019,may,5,41.033986,-73.762910,-79.383935,43.653482,0,11,11
425345,wichita,edmonton,canada,2019,may,5,37.692236,-97.337545,-113.507996,53.535411,140,0,140
425346,wichita,toronto,canada,2019,may,5,37.692236,-97.337545,-79.383935,43.653482,0,9,9


In [ ]:
#check whether data is present only for spring months
us_spring_df.month.unique()

array(['april', 'march', 'may'], dtype=object)

In [ ]:
#save season-wise data as separate csv file
us_summer_df.to_csv('intermittent-result/International US/us_summer_pax_data.csv', index=False)
us_autumn_df.to_csv('intermittent-result/International US/us_autumn_pax_data.csv', index=False)
us_winter_df.to_csv('intermittent-result/International US/us_winter_pax_data.csv', index=False)
us_spring_df.to_csv('intermittent-result/International US/us_spring_pax_data.csv', index=False)